## Домашнее задание к лекции "Основы веб-скрапинга и работы с API"
### Задание 1.
#### Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

#### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

In [8]:
import requests

In [9]:
from bs4 import BeautifulSoup

In [10]:
keywords = ['python', 'парсинг']

In [11]:
import pandas as pd

In [12]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [13]:
posts = soup.find_all('article', class_='post')

In [14]:
df_post = pd.DataFrame()
for post in posts:
    post_id = post.parent.attrs.get('id')
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    hubs = post.find_all('a', class_='hub-link')
    for hub in hubs:
            hub_lower = hub.text.lower()
            if any([hub_lower in desired for desired in keywords]):
                date_element = post.find('span', class_ = 'post__time')
                title_element = post.find('a', class_='post__title_link')
                text_element = post.find('div', class_ = 'post__text post__text-html post__text_v1')
                row = {'date': date_element.text, 'title': title_element.text, 'href': title_element.attrs.get('href'), 'text': text_element.text}
                df_post = pd.concat([df_post, pd.DataFrame([row])])
                break
            else:
                page = post.find('a', class_='btn btn_x-large btn_outline_blue post__habracut-btn')
                textpost = requests.get(page.attrs.get('href'))
                soup_text = BeautifulSoup(textpost.text, 'html.parser')
                text_posts = soup_text.find_all('div', class_='post__wrapper')
                for text in text_posts:
                    text_lower = text.text.lower()
                    for keywords_el in keywords:
                        if keywords_el in text_lower:
                            date_element = post.find('span', class_ = 'post__time')
                            title_element = post.find('a', class_='post__title_link')
                            text_element = post.find('div', class_ = 'post__text post__text-html post__text_v1')
                            row = {'date': date_element.text, 'title': title_element.text, 'href': title_element.attrs.get('href'), 'text': text_element.text}
                            df_post = pd.concat([df_post, pd.DataFrame([row])])
                            break
                break
df_post

,date,title,href,text
0,вчера в 17:45,Python и теория множеств,https://habr.com/ru/post/516858/,\nPython и теория множеств\nВ Python есть очен...


### Задание 2.
#### Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

#### Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.getGROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

In [124]:
email = ['a.yarmullin@gmail.com', 'a.yarmullid@gmail.com']
email_d ={}
import time
df_email = pd.DataFrame()
for el in email:
    email_d['email'] = el
    avast = requests.post('https://digibody.avast.com/v1/web/leaks', json = email_d)
    df_email2 = pd.DataFrame(avast.json()['value'])
    df_email = pd.concat([df_email, df_email2])
    time.sleep(0.2)
if df_email.empty:
    print('Утечек нет.')
else:
    df_email = df_email[['username', 'leak_date', 'leak_id', 'leak_info']]
df_email

Утечек нет.


""


In [49]:
soup.find?